## Challenge Part 2: have customers narrow their travel searches based on temperature and precipitation

In [2]:
import pandas as pd
import requests
import gmaps
from config import g_key

In [3]:
city_data_df = pd.read_csv("./data/WeatherPy_Database.csv")
city_data_df

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Rain (inches),Snow (inches),Wind Speed
0,0,Mataura,0,NZ,clear sky,83,-46.19,168.86,39.00,0.00,0,3.00
1,1,Aitape,60,PG,light rain,78,-3.14,142.35,83.70,0.71,0,6.44
2,2,Almonte,0,ES,clear sky,58,37.26,-6.52,72.00,0.00,0,6.15
3,3,Codrington,70,AU,light rain,80,-38.27,141.97,54.27,0.27,0,21.63
4,4,San Patricio,16,MX,few clouds,69,19.22,-104.70,80.80,0.00,0,2.98
...,...,...,...,...,...,...,...,...,...,...,...,...
558,558,Sistranda,75,NO,broken clouds,87,63.73,8.83,60.80,0.00,0,4.70
559,559,Maxixe,20,MZ,mist,93,-23.86,35.35,55.40,0.00,0,6.13
560,560,Margate,10,GB,clear sky,72,51.38,1.39,60.01,0.00,0,1.99
561,561,Leona Vicario,20,MX,few clouds,83,20.97,-87.19,82.40,0.00,0,6.93


In [4]:
city_data_df.dtypes

City_ID                  int64
City                    object
Cloudiness               int64
Country                 object
Current Description     object
Humidity                 int64
Lat                    float64
Lng                    float64
Max Temp               float64
Rain (inches)          float64
Snow (inches)            int64
Wind Speed             float64
dtype: object

In [5]:
# configure gmaps
gmaps.configure(api_key = g_key)

In [10]:
# Enter max temperature critieria
min_temp = float(input("What is the minimum temperature you would like for your vacation?"))
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation?60
What is the maximum temperature you would like for your vacation?80


In [11]:
# ask the customer if you want it to be raining
rain_amount = input("Do you want it to be raining? (yes/no)")

Do you want it to be raining? (yes/no)no


In [12]:
snow_amount = input("Do you want it to be snowing? (yes/no)")

Do you want it to be snowing? (yes/no)no


In [16]:
if rain_amount == "no" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] == 0)]
elif rain_amount == "no" and snow_amount == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] >= 0)]
elif rain_amount == "yes" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0.0) &
                                          (city_data_df["Snow (inches)"] == 0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0.0) &
                                          (city_data_df["Snow (inches)"] > 0)]

In [17]:
filtered_cities_df.head()

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Rain (inches),Snow (inches),Wind Speed
2,2,Almonte,0,ES,clear sky,58,37.26,-6.52,72.00,0.0,0,6.15
7,7,Rikitea,73,PF,broken clouds,67,-23.12,-134.97,70.34,0.0,0,17.60
8,8,Shar,0,KZ,clear sky,24,49.59,81.05,69.44,0.0,0,3.29
9,9,Tambura,99,SS,overcast clouds,91,5.60,27.47,72.36,0.0,0,1.41
10,10,Hilo,90,US,overcast clouds,61,19.73,-155.09,78.80,0.0,0,3.36


In [18]:
filtered_cities_df.count()

City_ID                230
City                   230
Cloudiness             230
Country                230
Current Description    230
Humidity               230
Lat                    230
Lng                    230
Max Temp               230
Rain (inches)          230
Snow (inches)          230
Wind Speed             230
dtype: int64

In [19]:
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Almonte,ES,72.00,37.26,-6.52,
7,Rikitea,PF,70.34,-23.12,-134.97,
8,Shar,KZ,69.44,49.59,81.05,
9,Tambura,SS,72.36,5.60,27.47,
10,Hilo,US,78.80,19.73,-155.09,


In [20]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["lacation"] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not Found! Skipping...")

Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Found! Skipping...
Hotel Not Foun

In [22]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Almonte,ES,72.00,37.26,-6.52,
7,Rikitea,PF,70.34,-23.12,-134.97,
8,Shar,KZ,69.44,49.59,81.05,
9,Tambura,SS,72.36,5.60,27.47,
10,Hilo,US,78.80,19.73,-155.09,


In [23]:
output_data_file = "./data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [24]:
vacation_df = pd.read_csv("./data/WeatherPy_VAcation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,2,Almonte,ES,72.00,37.26,-6.52,NaN
1,7,Rikitea,PF,70.34,-23.12,-134.97,NaN
2,8,Shar,KZ,69.44,49.59,81.05,NaN
3,9,Tambura,SS,72.36,5.60,27.47,NaN
4,10,Hilo,US,78.80,19.73,-155.09,NaN


In [25]:
# Using the template to add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt>
<dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat", "Lng"]]

In [26]:
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))